In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import os
from tqdm import tqdm

## Constructing a diet based on nutrient needs


### Diet diversity
Nutrients are contained in different quantities in different commodities; for example:
- roots and tubers contain a lot of vitamin A but a low amount of protein
- livestock and fish contain a lot of protein and vitamin B12
- some nutrients are generally harder to come by than others
A healthy diet requires food intake diversity.

### Malnutrition
The first cause of death by malnutrition is caused by a lack of protein. The most common mineral deficiency is iron deficiency.

Different regions have different diets and different deficiencies. In the USA for example, the nutrients that are under-consumed are fiber, calcium, vitamin D and potassium. However, in less-developped countries, there are a lot of iron and protein deficiencies.


### Meat vs. plants:
Plants are a fairly poor substitute for some of the nutrients meat provides. For example, iron and zinc contents in plants are very low. The main nutrients plants lack in high quantities are Vitamin A, Thiamin, Vitamin B12, zince and Calcium.

A diet relying more on plants rather than solely on meat is in accordance with most health councils and organizations. 
Vitamin B12 is mainly found in meat. Vitamin A is mainly found in butter, liver, and in carotenoids to some extent. Thiamin is found in meat and dairy, and cereals to a lesser extent. Calcium is mainly found in milk dairy, and cereals and vegetables to a much lesser extent. 


## Creating a nutrient list
These nutrients are deemed to be essential not only because of their paramount importance to a healthy diet but also because of the fact that there exists widespread deficiencies worldwide.

Some of these can be easily manufactured and distributed (e.g. iodine). We will only focus on a subset of nutrients, and look at protein content, fiber content, lipid content, vitamin content and mineral content of given foods in order to construct a "healthy" diet (meaning one that provides good nutrient intake).

- Proteins
- Carbohydrates
- Fibers
- Lipids
- Vitamins and minerals
    - Vitamins: A, B6, B9, B12, C, D, E
    - Minerals : Fe, Ca
    

# Nutrient value database
In parallel with the evaluation of the environmental impact of livestock and crops, we need to decide which food yield the best (or at least a good/healthy enough) nutritive value. 
To do this, after having decided on certain categories of nutrients (vitamins / fibers / etc...), we will check that the nutrients that we plan on using contain enough nutritive value using this database:

https://www.ars.usda.gov/northeast-area/beltsville-md-bhnrc/beltsville-human-nutrition-research-center/food-surveys-research-group/docs/fndds-download-databases/

Files:

Ingredient Nutrient Values.xlsx

In [133]:
food_nutrient = pd.read_excel("../2015-2016 FNDDS At A Glance - Ingredient Nutrient Values.xlsx", header = 1 )

In [134]:
food_nutrient.head(1)

,Ingredient code,SR description,Nutrient code,Nutrient description,Nutrient value,Nutrient value source,SR 28 derivation code,SR 28 AddMod year
0,1001,"Butter, salted",203,Protein,0.85,SR28,NaN,1976.0


In [135]:
# food_nutrient["SR description"].unique()
food_nutrient = food_nutrient.drop(["Ingredient code", "Nutrient code", "Nutrient value source", "SR 28 derivation code", "SR 28 AddMod year"], axis = 1)
food_nutrient.columns = ["ingredient","nutrient","value"]
# print(food_nutrient.ingredient.nunique(), food_nutrient.nutrient.nunique())

In [136]:
nutrient_list = list(food_nutrient.nutrient.unique())
selected_nutrients = ["Protein","Total Fat","Carbohydrate","Energy","Water",\
                      "Fiber, total dietary","Iron",\
                      "Calcium", "Vitamin C","Vitamin B-6",\
                      "Vitamin B-12","Vitamin A, RAE","Vitamin E (alpha-tocopherol)"\
                      ,"Vitamin D (D2 + D3)"]
assert(set(selected_nutrients) - set(nutrient_list) == set())

In [137]:
# # food_nutrient[food_nutrient.nutrient == "Calcium"].sort_values("value", ascending = False).head()
df = pd.DataFrame(columns = ["name","value","item"])
for idx, val in enumerate(selected_nutrients):
    max_val= food_nutrient[food_nutrient.nutrient == val].value.max()
    max_item = food_nutrient[food_nutrient.nutrient ==val].value.idxmax()
    max_item = food_nutrient.iloc[max_item]
    df.loc[idx] = val, max_val, max_item
df

,name,value,item
0,Protein,88.32,ingredient Soy protein isolate nutrient ...
1,Total Fat,100.00,"ingredient Fat, beef tallow nutrient ..."
2,Carbohydrate,100.00,"ingredient Sweetener, herbal extract powder..."
3,Energy,902.00,"ingredient Fat, beef tallow nutrient ..."
4,Water,99.98,"ingredient Water, bottled, generic nutrient..."
5,"Fiber, total dietary",53.20,"ingredient Spices, curry powder nutrient ..."
6,Iron,123.60,"ingredient Spices, thyme, dried nutrient ..."
7,Calcium,7364.00,"ingredient Leavening agents, baking powder,..."
8,Vitamin C,2400.00,"ingredient Fruit-flavored drink, powder, wi..."
9,Vitamin B-6,12.00,"ingredient Cereals ready-to-eat, KELLOGG, K..."


Obviously the dataset is biased by processed and enriched food which are made to contain more nutrients than "raw" foods. We will only focus on non-processed foods.

In [138]:
food_nutrient[(food_nutrient.ingredient.str.contains("meat"))&(food_nutrient.ingredient.str.contains("raw"))].ingredient.unique()

array(['Chicken, broiler or fryers, breast, skinless, boneless, meat only, raw',
       'Chicken, broilers or fryers, thigh, meat and skin, raw',
       'Duck, domesticated, meat only, raw',
       'Turkey, whole, meat and skin, raw',
       'Turkey, whole, meat only, raw',
       'Chicken, broilers or fryers, breast, skinless, boneless, meat only, enhanced, raw',
       'Turkey, retail parts, enhanced, breast, meat only, raw',
       'Turkey, retail parts, enhanced, breast, meat and skin, raw',
       'Pork, fresh, variety meats and by-products, stomach, raw',
       'Nuts, coconut meat, raw',
       'Nuts, coconut milk, raw (liquid expressed from grated meat and water)',
       'Beef, variety meats and by-products, tripe, raw',
       'Game meat, deer, ground, raw',
       'Beef, ground, 95% lean meat / 5% fat, raw',
       'Beef, ground, 90% lean meat / 10% fat, raw',
       'Beef, ground, 85% lean meat / 15% fat, raw',
       'Beef, ground, 80% lean meat / 20% fat, raw',
       'Be

We will only take a look at beef and chicken for clarity purposes. We will also only consider raw meat as the cooking process alters the nutrient values. We are going to average the different categories of raw meat we are interested in (e.g. breasts and legs for chicken, different fat contents for beef).

In [139]:
chicken = food_nutrient[(food_nutrient.ingredient.str.contains("meat"))&(food_nutrient.ingredient.str.contains("raw"))&(food_nutrient.ingredient.str.contains("Chicken"))].reset_index()
beef = food_nutrient[(food_nutrient.ingredient.str.contains("meat"))&(food_nutrient.ingredient.str.contains("raw"))&(food_nutrient.ingredient.str.contains("Beef"))].reset_index()
chicken = chicken[chicken.nutrient.isin(selected_nutrients)].groupby(["nutrient"]).agg({"value":"mean"}).reset_index().rename({"value":"chicken_value"},axis=1)
beef = beef[beef.nutrient.isin(selected_nutrients)].groupby(["nutrient"]).agg({"value":"mean"}).reset_index().rename({"value":"beef_value"},axis=1)
both_meats = chicken.merge(beef, on = "nutrient").sort_values("nutrient")

both_meats.style.background_gradient(cmap='Reds', low=0.2, high=0.6, axis =1,)

,nutrient,chicken_value,beef_value
0,Calcium,5.66667,24
1,Carbohydrate,0.0833333,0
2,Energy,149.667,193.333
3,"Fiber, total dietary",0,0
4,Iron,0.463333,1.83833
5,Protein,19.78,17.5
6,Total Fat,7.41,13.115
7,"Vitamin A, RAE",14.3333,3.33333
8,Vitamin B-12,0.34,2.04333
9,Vitamin B-6,0.639667,0.290833


We can see that chicken is lacking the Calcium, Iron and Vitamin B-12 when compared to beef. Therefore we would need to find a complementary substitute to replace beef with chicken. Fiber and Vitamin C are contained within crops and should not be considered here.

In [140]:
food_nutrient[((food_nutrient.ingredient.str.contains("Milk, whole"))&(food_nutrient.ingredient.str.contains("without")))&(food_nutrient.nutrient.str.contains("Calcium"))]

,ingredient,nutrient,value
7030,"Milk, whole, 3.25% milkfat, without added vita...",Calcium,113.0


Milk is a good substitute for Calcium.

In [141]:
food_nutrient[((food_nutrient.ingredient.str.contains("Lentils, raw"))|(food_nutrient.ingredient.str.contains("Cereals ready-to-eat, granola, homemade")))&((food_nutrient.nutrient.str.contains("Iron")))]

,ingredient,nutrient,value
46031,"Cereals ready-to-eat, granola, homemade",Iron,3.95
111031,"Lentils, raw",Iron,6.51


Granola is made from different cereals (here: wheat) and oats and is a good substitute for Iron deficiencies. Lentils are another good substitute. It is harder to find Vitamin B-12 ingredients that could replace beef.

In [143]:
food_nutrient[(food_nutrient.nutrient == "Vitamin B-12")\
              &~(food_nutrient.ingredient.str.contains("Beef"))\
              &~(food_nutrient.ingredient.str.contains("Cereals"))]\
                .sort_values("value", ascending = False).head(10)

,ingredient,nutrient,value
109299,"Mollusks, octopus, common, cooked, moist heat",Vitamin B-12,36.00
41569,"Braunschweiger (a liver sausage), pork",Vitamin B-12,20.09
104879,"Fish, caviar, black and red, granular",Vitamin B-12,20.00
108844,"Mollusks, octopus, common, raw",Vitamin B-12,20.00
108974,"Mollusks, oyster, eastern, canned",Vitamin B-12,19.13
105854,"Fish, herring, Atlantic, kippered",Vitamin B-12,18.70
108649,"Mollusks, clam, mixed species, canned, drained...",Vitamin B-12,18.63
29934,"Chicken, liver, all classes, cooked, simmered",Vitamin B-12,16.85
29869,"Chicken, liver, all classes, raw",Vitamin B-12,16.58
31689,"Turkey, whole, giblets, cooked, simmered",Vitamin B-12,15.89


It will be hard to replace the Vitamin B-12 found in beef-based diets, but there are alternatives (fish, game meat, chicken liver).
Overall we have seen that it is viable to replace beef consumption over the world from a nutrition and diet point of view.

In [144]:
potatoes = food_nutrient[food_nutrient.ingredient.str.contains("Potatoes")&(food_nutrient.ingredient.str.contains("raw"))]
potatoes = potatoes[potatoes.nutrient.isin(selected_nutrients)].rename({"value":"potato_value"},axis = 1).drop("ingredient",axis = 1)
rice = food_nutrient[food_nutrient.ingredient.str.contains("Rice, brown")&(food_nutrient.ingredient.str.contains("raw"))]
rice = rice[rice.nutrient.isin(selected_nutrients)].rename({"value":"rice_value"},axis = 1).drop("ingredient", axis = 1)
lentils = food_nutrient[food_nutrient.ingredient.str.contains("Lentils, raw")]
lentils = lentils[lentils.nutrient.isin(selected_nutrients)].rename({"value":"lentils_value"},axis = 1).drop("ingredient", axis = 1)
both_crops = rice.merge(potatoes, on = "nutrient").merge(lentils, on = "nutrient").sort_values("nutrient")
both_crops.style.background_gradient(cmap='Reds', low=0, high=1, axis =1,)

,nutrient,rice_value,potato_value,lentils_value
6,Calcium,9,12,35
2,Carbohydrate,76.25,17.49,63.35
3,Energy,367,77,352
5,"Fiber, total dietary",3.6,2.1,10.7
7,Iron,1.29,0.81,6.51
0,Protein,7.54,2.05,24.63
1,Total Fat,3.2,0.09,1.06
8,"Vitamin A, RAE",0,0,2
13,Vitamin B-12,0,0,0
12,Vitamin B-6,0.477,0.298,0.54


We can see that while potatoes are not a good enough nutrition substitute, lentils are a good substitute. 
Therefore it is reasonable to say that rice and beef can be compensated for by other meats or crops from a nutritional standpoint.

# References:

https://www.intechopen.com/books/meat-science-and-nutrition/nutritional-composition-of-meat

https://www.sciencedirect.com/science/article/pii/S2542519617300074

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2442550/

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3546624/

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6213018/

https://www.ncbi.nlm.nih.gov/pubmed/28532520

http://www.fao.org/3/T0207E04.htm

https://academic.oup.com/ajcn/article/78/3/660S/4690010

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5346408/

http://www.fao.org/3/ca6030en/ca6030en.pdf

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3546624/

https://en.wikipedia.org/wiki/Rice#Nutrition

https://www.earth.columbia.edu/articles/view/3281

https://www.epa.gov/ghgemissions/sources-greenhouse-gas-emissions